# การใช้งาน Transaction Trino
---
> ### Function นี้รองรับเฉพาะ Hive >= 3 เท่านั้น
## Step Hive Enable ACID Transactions
#### Hive-site.xml Setting (Ambari or Cloudera)
* Create Tables as ACID Insert Only = False (On Ambari)
* hive.support.concurrency = true
* hive.enforce.bucketing = true
* hive.create.as.external.legacy = true
* hive.exec.dynamic.partition.mode = nonstrict
* hive.txn.manager = org.apache.hadoop.hive.ql.lockmgr.DbTxnManager
* Trino Catalog > /home/trino/etc/catalog/hive/properties <br>
Add Above Line `hive.storage-format=ORC`
---
## สร้าง Table ที่ Trino
---
```sh
CREATE TABLE hive.default.covid (
	year int,
	weeknum int,
	province varchar,
	new_case int,
	total_case int,
	new_case_excludeabroad int,
	total_case_excludeabroad int,
	new_death int,
	total_death int,
	update_date varchar,
    idx
)
WITH (
  format = 'ORC',
  transactional=true,
  partitioned_by = ARRAY['idx'],
  bucketed_by = ARRAY['province'],
  bucket_count = 5
);
```
## Install Libraly
---
```bash
pip install pandas==2.0.0rc0
pip install trino
pip install trino[sqlalchemy]
```

In [ ]:
import pandas as pd
from trino.dbapi import connect
import os
from sqlalchemy import create_engine
from sqlalchemy.schema import Table, MetaData
from sqlalchemy.sql.expression import select, text

> ### Set Default Backend Pandas to Arrow

In [ ]:
pd.options.mode.dtype_backend = 'pyarrow'

> ### Trino Connect with SQLAlchemy, Trino API

In [ ]:
engine = create_engine('trino://hive:@192.168.10.210:8090/hive/transaction')
cur = engine.connect()

In [ ]:
df = pd.read_json("https://covid19.ddc.moph.go.th/api/Cases/today-cases-by-provinces")

> ### Finding Max ID For insert using to Partitioned ORC

In [ ]:
max_id = pd.read_sql("SELECT MAX(id) as max_id FROM hive.transaction.covid", con=engine)

> ### Loop inserting row-by-row Using Transaction

In [ ]:
i = max_id['max_id'].astype(int).iloc[0]
for row in df.itertuples():
    i = i + 1
    sql = f"""
    INSERT INTO
    	hive.transaction.covid
	VALUES (
    	{row.year},
        {row.weeknum},
        '{row.province}',
        {row.new_case},
        {row.total_case},
        {row.new_case_excludeabroad},
        {row.total_case_excludeabroad},
        {row.new_death},
        {row.total_death},
        '{row.update_date}',
        {i}
        )
    """
    cur.execute(sql)
    print('Insert Rows :', row.province, 'Number of Insert:', i)

> ### Update

In [ ]:
cur.execute("UPDATE default.covid SET province = 'Bangkok City' WHERE province = 'กรุงเทพมหานคร'")

> ### Delete

In [ ]:
cur.execute("DELETE FROM transaction.covid")